In [1]:
import pandas as pd
import gpxpy
from geopy.distance import geodesic

In [2]:
gpx_file = open('../data/SG200Miles2024.gpx', 'r')
gpx = gpxpy.parse(gpx_file)

In [3]:
data = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            data.append({'latitude': point.latitude, 'longitude': point.longitude})

In [4]:
df = pd.DataFrame(data)

df['prev_latitude'] = df['latitude'].shift(1)
df['prev_longitude'] = df['longitude'].shift(1)

df = df.dropna().reset_index(drop=True)

In [7]:
df['distance'] = df.apply(lambda x: geodesic((x['prev_latitude'], x['prev_longitude']), (x['latitude'], x['longitude'])).meters, axis=1)
df['accumulated_distance'] = df['distance'].cumsum() / 1000.0

In [8]:
df.head(3)

,latitude,longitude,prev_latitude,prev_longitude,distance,accumulated_distance
0,1.34265,103.83574,1.34228,103.83519,73.616069,0.073616
1,1.34276,103.83592,1.34265,103.83574,23.438023,0.097054
2,1.34300,103.83631,1.34276,103.83592,50.877286,0.147931


In [10]:
df.to_csv(
    '../postgis/data/gpx-route.csv', 
    columns=['latitude','longitude','accumulated_distance'],
    header=False,
    index=False,
)